<a href="https://colab.research.google.com/github/pranilbasu/ML-Projects/blob/main/DL_Project_on_Decision_of_Bank_Customer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction of decision of Bank Customer

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
dataset = pd.read_csv('Churn_Modelling.csv') 

In [ ]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


##### We shall predict whether any bank customer will exit the bank in future 

In [ ]:
X = dataset.iloc[:,3 :-1].values  # matrix of features (Customer ID, Surname have been excluded)
Y = dataset.iloc[:,-1].values     # dependent variables

In [ ]:
# Encoding Gender feature
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

In [ ]:
# Encoding Geography feature
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [ ]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Building the ANN 

In [ ]:
ann = tf.keras.models.Sequential()  # intitializing ANN

In [ ]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu')) # Adding the input layer and first hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu')) # Adding the second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))  #Adding the output layer

In [ ]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) # Compiling the ANN

#### Training the ANN

In [ ]:
ann.fit(X_train, Y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 0s 510us/step - loss: 0.6457 - accuracy: 0.6607
Epoch 2/100
250/250 [==============================] - 0s 473us/step - loss: 0.4944 - accuracy: 0.7993
Epoch 3/100
250/250 [==============================] - 0s 493us/step - loss: 0.4515 - accuracy: 0.8087
Epoch 4/100
250/250 [==============================] - 0s 529us/step - loss: 0.4373 - accuracy: 0.8111
Epoch 5/100
250/250 [==============================] - 0s 533us/step - loss: 0.4310 - accuracy: 0.8136
Epoch 6/100
250/250 [==============================] - 0s 553us/step - loss: 0.4266 - accuracy: 0.8160
Epoch 7/100
250/250 [==============================] - 0s 534us/step - loss: 0.4238 - accuracy: 0.8175
Epoch 8/100
250/250 [==============================] - 0s 510us/step - loss: 0.4213 - accuracy: 0.8196
Epoch 9/100
250/250 [==============================] - 0s 541us/step - loss: 0.4195 - accuracy: 0.8211
Epoch 10/100
250/250 [==============================] - 0s 511us/step - l

#### Prediction of a new result

In [ ]:
# Geography : Spain (001) | Credit Score: 600 | Gender: Female (0) | Age: 40 | Tenure:3(years) | Balance:50000 
# Number of Products: 2 | Credict Card: No(0) | Active Member:Yes(1) | Salary : 60000

# Will the customer exit the bank ?

print(ann.predict(sc.transform([[0,0,1, 600, 0, 40, 3, 50000, 2, 0, 1, 60000]]))>0.45)  #if  the probability is greater than 0.45, result is True or else False 

[[False]]


In [ ]:
#Predicting test set results
Y_pred = ann.predict(X_test)   
Y_pred = (Y_pred > 0.45)
print(np.concatenate((Y_pred.reshape(len(Y_pred),1), Y_test.reshape(len(Y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


#### Testing the ANN model

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(Y_test, Y_pred)
print(cm)

[[1489  106]
 [ 186  219]]


In [ ]:
accuracy_score(Y_test, Y_pred)*100

85.39999999999999